# Projecto Integrador

Primer acercamiento a trabajar con el concepto de web Scrapping. El proyecto fué propuesto en clase por el maestro Gonzalo del Rio, durante el primer módulo del Bootcamp de Henry.

El proyecto consiste en hacer WebScrapping al sitio web https://cuspide.com/ para extraer la información de la sección de "100 libros más vendidos de la semana" en un formato permita hacer el procesamiento con Pandas y crear finalmente un archivo csv de salida.

***

### Para empezar:

#### 1) Preparar el ambiente. 

Se importan las librerías de procesamiento de los datos con las que se va a trabajar.

- Se usa Pandas para el procesamiento y visualización de los datos. **Se requiere crear un DataFrame.**
- Se usa el módulo `requests` para enviar requerimentos HTTP al sitio que nos interesa. Como respuesta queremos obtener el HTML crudo de la página.
- Se usa `Beautiful Soup` para leer el HTML en una estructura que sea útil para extraer la información.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import requests
from bs4 import BeautifulSoup

In [3]:
# Define a variable containing the website's url.
url = 'https://cuspide.com/100-mas-vendidos/' 

# Making the HTTP request using request.get() method.
# A response object is returned.
response = requests.get(url) 

# Getting raw html from response.content attribute.
html_content = response.content

### 2) BeautifulSoup 

Beautiful soup interpreta el contenido html crudo y lo organiza en estructura jerárquica, parecido a como lo hace el navegador. Con la estructura de etiquetas donde podemos analizar esa jerarquía ya se puede empezar a estraer la información. 

Esa es la magia de Beautiful soup :D

In [4]:
# Parsing the content with bs4
soup = BeautifulSoup(html_content, 'html.parser')

Luego de inspeccionar la página, se observó que los titulos están contenidos en una etiqueta `<h3>` con el atributo de clase "product-title".

**Nota**: El método `find_all()` retorna un objeto tipo ResultSet que contiene todas las etiquetas que cumplan con los parámetros que se pasen por la función. Este ResultSet se puede iterar para trabajar con cada etiqueta específica.

In [24]:
# Finding title's tags for each book
title_tags = soup.find_all("h3", class_='product-title')

for tag in title_tags[:5]: # Loop over each tag
   print(tag.get_text(strip=True)) # Printing text within every tag

LA MUJER QUE SOY
ARTIFICIAL
ESTE DOLOR NO ES MIO
LA CASA NEVILLE . LA FORMIDABLE SEÑORITA  MANON
OXIDO


Teniendo las etiquetas, podemos guardarlas en una lista (usando list comprehension).

In [25]:
# Making a list with titles
titulos = [tag.get_text(strip=True).title() for tag in title_tags]
titulos[:5]

['La Mujer Que Soy',
 'Artificial',
 'Este Dolor No Es Mio',
 'La Casa Neville . La Formidable Señorita  Manon',
 'Oxido']


... Por inspección tambíen puede verse que los url de cada libro están contenidos en una etiqueta "hija" dentro de las mismas etiquetas de los títulos.

**Nota**: El objeto etiqueta nos permite acceder a las etiquetas que tiene por dentro (estructura jerárquica de arbol). Con el atritubo .attrs podemos ver un diccionario con los atributos de la etiqueta.

In [26]:
# Making a list with urls for every book detail
urls = [tag.a.attrs['href'] for tag in title_tags]
urls[:5]

['https://cuspide.com/producto/la-mujer-que-soy/',
 'https://cuspide.com/producto/artificial-2/',
 'https://cuspide.com/producto/este-dolor-no-es-mio/',
 'https://cuspide.com/producto/la-casa-neville-la-formidable-senorita-manon/',
 'https://cuspide.com/producto/oxido/']

#### Para los precios se usó un método de distinto (para ilustrar que hay diferentes formas de hacerlo).

En lugar de find_all, puede usarse el método selector de css que de igual manera es muy cómodo para seleccionar las etiquetas por clases, estilos y id.

In [27]:
# Selecting tags by css classes
price_tags = soup.css.select('.product-price')

for tag in price_tags[:5]:
    print(tag.get_text(strip=True))

$8.999,00
$9.999,00
$11.500,00
$13.700,00
$14.999,00


In [28]:
# Making list for prices. Getting only numeric part.
price_column = [tag.get_text(strip=True)[1:] for tag in price_tags]

# Formatting and casting to float.
price_column = [float(price.replace(".","").replace(",",".")) for price in price_column]
price_column[:5]

[8999.0, 9999.0, 11500.0, 13700.0, 14999.0]

Debido a que el precio en dólares se encuentra en cada url específico para cada libro, he creado funciones scrapper para esta tarea.
Las funciones están definidas dentro del módulo `functions.py`. Allí puede verse el script que se ha creado.

In [10]:
from functions import get_usdprice_and_date

# Getting usd price for each book.
usd_prices = []
dates = [] 
for url in urls:
    usd_price, date = get_usdprice_and_date(url)
    usd_prices.append(usd_price)
    dates.append(date)

In [11]:
usd_prices[:5], dates[:5]

(['24,62', '27,36', '31,46', '37,48', '41,04'],
 ['31/10/2023', '28/09/2023', '16/03/2018', '29/09/2023', '06/10/2023'])

In [12]:
# Formatting and casting usd prices
usd_prices = [float(price.replace(",",".")) for price in usd_prices]
usd_prices[:5]

# Formatting dates to SQL format
from functions import format_date
dates = [format_date(date, old_format='%d/%m/%Y', new_format='%Y-%m-%d') for date in dates]

usd_prices[:5], dates[:5]

([24.62, 27.36, 31.46, 37.48, 41.04],
 ['2023-10-31', '2023-09-28', '2018-03-16', '2023-09-29', '2023-10-06'])

Obteniendo el precio del Dolar Blue:

In [13]:
r = requests.get('https://www.cronista.com/MercadosOnline/dolar.html')
dom_content = r.text
html_soup = BeautifulSoup(dom_content, 'html.parser')

dolar_blue = html_soup.find('span', string='Dólar Blue')

In [15]:
# Span tag containing value is the next one. "The sibling"
dolar_blue.next_sibling.text[1:]
# Formatting and casting.
dolarBlue_precio = float(dolar_blue.next_sibling.text[1:].replace(",","."))
dolarBlue_precio

925.0

In [16]:
# Making list with dolar blue conversion.
blue_column = [round(precio / dolarBlue_precio, 2) for precio in price_column]
blue_column[:5]

[9.73, 10.81, 12.43, 14.81, 16.22]

***
### Creación del archivo de salida:

- The following step is to create a .csv output file.
- Using csv module for this purpose.

In [21]:
table = []
for i in range(len(title_tags)):
    table.append(
        (titulos[i], dates[i], urls[i], price_column[i], usd_prices[i], blue_column[i])
    )

# Now each tuple is the equivalent to a row
table[:3]

[('La Mujer Que Soy',
  '2023-10-31',
  'https://cuspide.com/producto/la-mujer-que-soy/',
  8999.0,
  24.62,
  9.73),
 ('Artificial',
  '2023-09-28',
  'https://cuspide.com/producto/artificial-2/',
  9999.0,
  27.36,
  10.81),
 ('Este Dolor No Es Mio',
  '2018-03-16',
  'https://cuspide.com/producto/este-dolor-no-es-mio/',
  11500.0,
  31.46,
  12.43)]

In [23]:
# Writing the csv file
with open('./libros_semana.csv', 'w') as file:
    w = csv.writer(file, delimiter=',') # Writer object
    w.writerow(('titulo', 'fecha_publicación', 'url', 'precio_pesos', 'precio_usd', 'precio_dolar_blue'))
    w.writerows(table)

*** 
2da Parte del proyecto - Módulo 3

## Conexión con MySQL: Creando una base de datos.

En esta sección se crea una base de datos MySQL en lugar de un archivo csv. También se implementa manejo de excepciones para crear una tabla de errores cuando los haya.

In [ ]:
import pymysql

In [ ]:
# Creating connection
connection = pymysql.Connection(
    host='localhost',
    user='juancml',
    passwd='',
    database='libros'
)

# creating mysql cursor
cursor = connection.cursor()
# Query to insert values on mysql table.
query = """ INSEERT INTO libros_semana 
            VALUES (%s,%s,%s,%s,%s,%s, CURRDATE()) """

Se hace la insercion de los datos con el método del cursor `executemany()`.

In [ ]:
cursor.executemany(query, table)
# Commit changes and close database connection
connection.commit()
connection.close()